<a href="https://colab.research.google.com/github/mohanrajmit/AgeCalculation/blob/master/Agecalculation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Age Estimation for Indian Movie Face Dataset like Young, Old and Middle

In [1]:
!git clone https://github.com/mohanrajmit/AgeCalculation.git

fatal: destination path 'AgeCalculation' already exists and is not an empty directory.


In [2]:
# keras imports

from keras.applications.xception import Xception, preprocess_input
from keras.applications import xception
from keras.preprocessing import image
from keras.models import Model
from keras.models import model_from_json
from keras.layers import Input
from keras.applications.inception_resnet_v2 import InceptionResNetV2

# other imports
from sklearn.preprocessing import LabelEncoder
import numpy as np
import glob
import cv2
import h5py
import os
import json
import datetime
import time
import pandas as pd




In [3]:
data=pd.read_csv("AgeCalculation/dataset/train.csv")

image_name = data["ID"]
label_name = data["Class"]

features = []
labels = []

In [4]:
print(data.head())

          ID   Class
0    377.jpg  MIDDLE
1  17814.jpg   YOUNG
2  21283.jpg  MIDDLE
3  16496.jpg   YOUNG
4   4487.jpg  MIDDLE


In [5]:
model = xception.Xception(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

image_size = (299,299)

In [6]:
dataset_path="AgeCalculation/dataset/Train/"

In [ ]:
for image_name1,label in zip(image_name,label_name):
	
	print("the image name  {} and label{}" .format(image_name1,label))
	image_path= str(dataset_path+image_name1)
	#print(image_path)
	#image =cv2.imread(image_path)
	#images.append(image)
	#labels.append(label)

	img = image.load_img(image_path, target_size=image_size)
	x = image.img_to_array(img)
	x = np.expand_dims(x, axis=0)
	x = preprocess_input(x)
	feature = model.predict(x)
	flat = feature.flatten()
	features.append(flat)
	labels.append(label)

Streaming output truncated to the last 5000 lines.
the image name  1717.jpg and labelYOUNG
the image name  24505.jpg and labelYOUNG
the image name  17159.jpg and labelMIDDLE
the image name  2350.jpg and labelYOUNG
the image name  4658.jpg and labelOLD
the image name  8164.jpg and labelMIDDLE
the image name  18393.jpg and labelMIDDLE
the image name  22559.jpg and labelMIDDLE
the image name  10872.jpg and labelOLD
the image name  22155.jpg and labelMIDDLE
the image name  21487.jpg and labelMIDDLE
the image name  13909.jpg and labelMIDDLE
the image name  10839.jpg and labelMIDDLE
the image name  17399.jpg and labelMIDDLE
the image name  9687.jpg and labelYOUNG
the image name  24904.jpg and labelYOUNG
the image name  10653.jpg and labelMIDDLE
the image name  22622.jpg and labelMIDDLE
the image name  3985.jpg and labelOLD
the image name  18705.jpg and labelOLD
the image name  14381.jpg and labelMIDDLE
the image name  22435.jpg and labelMIDDLE
the image name  9892.jpg and labelOLD
the image 

In [ ]:
# encode the labels using LabelEncoder
le = LabelEncoder()
le_labels = le.fit_transform(labels)

# get the shape of training labels
print ("[STATUS] training labels: {}".format(le_labels))
print ("[STATUS] training labels shape: {}".format(le_labels.shape))

# save features and labels
h5f_data = h5py.File("features.h5", 'w')
h5f_data.create_dataset('dataset_1', data=np.array(features))

h5f_label = h5py.File("label.h5", 'w')
h5f_label.create_dataset('dataset_1', data=np.array(le_labels))

h5f_data.close()
h5f_label.close()

In [ ]:
list(le.inverse_transform([0, 1, 2]))

In [ ]:
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
import numpy as np
import h5py
import os
import json
import pickle
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from __future__ import print_function

In [ ]:
# import features and labels
h5f_data  = h5py.File("features.h5", 'r')
h5f_label = h5py.File("label.h5", 'r')

features_string = h5f_data['dataset_1']
labels_string   = h5f_label['dataset_1']

features = np.array(features_string)
labels   = np.array(labels_string)

h5f_data.close()
h5f_label.close()

# verify the shape of features and labels
print ("[INFO] features shape: {}".format(features.shape))
print ("[INFO] labels shape: {}".format(labels.shape))

print ("[INFO] training started...")
# split the training and testing data

In [ ]:
(trainData, testData, trainLabels, testLabels) = train_test_split(np.array(features),
                                                                  np.array(labels),
                                                                  test_size=0.10,
                                                                  random_state=9)

print("[INFO] splitted train and test data...")
print("[INFO] train data  : {}".format(trainData.shape))
print("[INFO] test data   : {}".format(testData.shape))
print("[INFO] train labels: {}".format(trainLabels.shape))
print("[INFO] test labels : {}".format(testLabels.shape))



Logisitic Regression Algorithm to train the pretrained model extracted features.

In [ ]:
# use logistic regression as the model
print("[INFO] creating model...")
model_ml = LogisticRegression(random_state=9)
model_ml.fit(trainData, trainLabels)

print("[INFO] evaluating model...")
f = open("results.txt", "w")

	
# evaluate the model of test data
preds = model_ml.predict(testData)

# write the classification report to file
f.write("{}\n".format(classification_report(testLabels, preds)))
f.close()

classifier_path="classifier.pickle"

# dump classifier to file
print("[INFO] saving model...")
pickle.dump(model_ml, open(classifier_path, 'wb'))



In [ ]:
!rm -r AgeCalculation

In [ ]:
!git clone https://github.com/mohanrajmit/AgeCalculation.git

In [ ]:
test_dataset="AgeCalculation/dataset/Test/"
#!pip install imutils

In [ ]:
data_test=pd.read_csv("AgeCalculation/dataset/test.csv")

image_name_test = data_test["ID"]

features = []
labels_test = []

In [ ]:
print(len(image_name_test))

In [ ]:
base_model = Xception(weights="imagenet")
model = Model(input=base_model.input, output=base_model.get_layer('avg_pool').output)
image_size = (299,299)

In [ ]:
images_label =['MIDDLE', 'OLD', 'YOUNG']
images_name = []
labels_test = []
for image_name1 in image_name_test:
  
  image_pa = test_dataset+image_name1
  print(image_pa)
  
  img = image.load_img(image_pa, target_size=image_size)
  #image_list = image_name1.split("/")[-1]
  print(image_pa)
  #make = imagePath.split("/")[-2]
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x = preprocess_input(x)
  feature = model.predict(x)
  flat = feature.flatten()
  flat = flat.reshape(1, -1)
  
  preds = model_ml.predict(flat)
  
 
  preds= images_label[int(preds)]
  print(preds)
  labels_test.append(preds)
  images_name.append(image_name1)

In [ ]:
print(np.array(images_name).shape)
print(np.array(labels_test).shape)

In [ ]:
sub = pd.DataFrame({'Class': labels_test, 'ID': images_name})
sub.to_csv('output.csv', index=False)